In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
sns.set()

from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import tensorflow as tf

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanictestdata/titanicFinalTestData.npz
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanicdata3/titanicTrainData.npz
/kaggle/input/titanicdata3/titanicValidationData.npz
/kaggle/input/titanicdata3/titanicTestData.npz


# Get data

In [33]:
address = '/kaggle/input/titanicdata3/'
address2 = '/kaggle/input/titanictestdata/'

npz = np.load(address + 'titanicTrainData.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load(address + 'titanicValidationData.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load(address + 'titanicTestData.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load(address2 + 'titanicFinalTestData.npz')
finalTestInputs = npz['inputs'].astype(np.float)

# Model

## Outline, optimizers, loss, early stopping and training

In [41]:
input_size = 9
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                             ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 10
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=4)

model.fit(
    train_inputs, 
    train_targets, 
    batch_size = batch_size, 
    epochs = max_epochs,
    callbacks = [early_stopping], 
    validation_data = (validation_inputs, validation_targets), 
    verbose = 2
    )

Train on 712 samples, validate on 89 samples
Epoch 1/100
712/712 - 1s - loss: 0.6828 - accuracy: 0.5885 - val_loss: 0.6523 - val_accuracy: 0.6742
Epoch 2/100
712/712 - 0s - loss: 0.6396 - accuracy: 0.6657 - val_loss: 0.5764 - val_accuracy: 0.7978
Epoch 3/100
712/712 - 0s - loss: 0.5712 - accuracy: 0.7753 - val_loss: 0.4826 - val_accuracy: 0.8427
Epoch 4/100
712/712 - 0s - loss: 0.5034 - accuracy: 0.8062 - val_loss: 0.4281 - val_accuracy: 0.8315
Epoch 5/100
712/712 - 0s - loss: 0.4770 - accuracy: 0.8076 - val_loss: 0.4160 - val_accuracy: 0.8202
Epoch 6/100
712/712 - 0s - loss: 0.4650 - accuracy: 0.7893 - val_loss: 0.3962 - val_accuracy: 0.8539
Epoch 7/100
712/712 - 0s - loss: 0.4635 - accuracy: 0.8034 - val_loss: 0.3947 - val_accuracy: 0.8427
Epoch 8/100
712/712 - 0s - loss: 0.4471 - accuracy: 0.8090 - val_loss: 0.4094 - val_accuracy: 0.8539
Epoch 9/100
712/712 - 0s - loss: 0.4480 - accuracy: 0.8104 - val_loss: 0.3882 - val_accuracy: 0.8427
Epoch 10/100
712/712 - 0s - loss: 0.4423 - acc

# Test the model

In [6]:
test_inputs.shape

(90, 9)

In [7]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

90/90 [==============================] - 0s 82us/sample - loss: 0.3857 - accuracy: 0.8333


# Test the model on new data

In [8]:
predictions = model.predict(finalTestInputs)

In [9]:
predictions.shape

(418, 2)

# Prepare results to deploy on kaggle

In [10]:
df = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [14]:
survived = predictions[:, 1]

response = []

for i in survived:
    if i >= 0.5:
        i = 1
        response.append(int(i))
    else:
        i = 0
        response.append(int(i))
labels = pd.DataFrame(response)
labels

,0
0,1
1,1
2,1
3,1
4,1
...,...
413,1
414,0
415,1
416,1


In [15]:
df['Survived'] = labels

# Save data to csv

In [16]:
df.to_csv(r'finalTitanicResponse.csv', index = False)